## Implementation Workflow

##### ■ Efficient implementation of the sweeps

- Adjust the `square_mpo` methods to the needs of the `XTRG_update` function 
- The `XTRG_step` function should update $\rho(\beta)$ to $\rho(2 \beta)$ if possible and otherwise throw an error which aborts the simulation.
- After a number of `Nsweeps` iterations, the overlap $||\rho(2\beta)- \rho_{\text{opt}}(2\beta)||_F^2$ is computed to check whether the optimization has sufficiently converged as specified by the `tolerance` value. 
- Compute the inner product of the $\rho(2\beta)_{\text{diff}}$ operator while carefully performing the correct trace. The function should print a message to track that the result is still compatible with the `tolerance`. In this way, be can later adjust the number of `Nsweeps` that is appropriate for our simulation purposes.
- At the beginning of the step, we left-canonicalize with `leftcanonicalmpo` from MPO.jl
- As we work with canonicalized MPOs, the result of contraction in Fig. 17a of Appendix D in the main paper is exactly the optimal two-site tensor. Applying the SVD on this tensor, we easily obtain the new tensors $C_i$ and $C_{i+1}$ to update our state $\rho(2\beta)$.
- To start the sweeping it may be advantageous to compute all the left environments in advance. (Damiano agrees!) In the process of sweeping from right to left, we may then delete one left environment in the environment array `Envs` and replace it with the new right environment step by step. This should be similar to the data structure in the 2-site DMRG algorithm.  There, in particular, `Hlr[l + 1]` contains the left/right environment tensor relative to site `l` when the algorithm/sweeping is working at a couple of site that are on the right/left of site `l`, respectively (watch the indices!)
- Following the 2-site DMRG algorithm, i.e. the implementation/indexing/... of `DMRG_2site`, is a good idea also for implementing the sweeps. There, the sweeping works like this: <br>
-- Start a for cycle with the index `itS = (1:Nsweeps)` <br>
-- RIGHT TO LEFT SWEEP: `for itL = (L:-1:2)`, update as in appendix D of main ref. `mpo2[itL-1]` and `mpo2[itL]`. This involves `Vlr[itL-1]` as left environment $V_L$ and `Vlr[itL + 2]` as right environment $V_R$. It also involves an SVD to get `mpo2[itL-1]` and `mpo2[itL]` out of the 2-site optimized tensor ($C_i C_{i+1}$ in the paper). In particular, the updated `mpo2[itL]` must be an "MPO-right isometry", so that the updated `mpo2`  will be "MPO-site canonical" with new isometry center at site `itL - 1`. After that, update `Vlr[itL + 1]` using `updateLeft_mod` (and an index permutation since we are "updating leftwards")<br>
-- LEFT TO RIGHT SWEEP: similar to above: `for itL = (1:L-1)`, update `mpo2[itL]` and `mpo2[itL+1]` using `Vlr[itL] ` and `Vlr[itL + 3]`. Then update `Vlr[itL + 1]` using `updateLeft_mod`<br>
-- end of cycle with index itS
- Understand how contracting the SVD decomposed tensors preserves the site-canonical form of the MPO for subsequent updates.